# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [147]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [148]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rikitea,-23.1203,-134.9692,76.95,69,24,12.08,PF,1681314365
1,1,isiro,2.7739,27.6160,79.38,60,100,3.15,CD,1681314365
2,2,sitka,57.0531,-135.3300,39.09,65,100,8.05,US,1681314365
3,3,yellowknife,62.4560,-114.3525,16.32,73,75,11.50,CA,1681314366
4,4,ahipara,-35.1667,173.1667,64.40,87,25,12.97,NZ,1681314366


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [150]:
%%capture --no-display

# Configure the map plot
humidity = city_data_df.hvplot.points(
"Lng",    
"Lat",
 gewopoints=True,
title="Humidity",
frame_width = 600,  
frame_height = 360,
size=2,
scale=2,
color="Humidity",
 hover_cols=["City", "Country",  "Humidity"]
)
   
    

# Display the map
humidity


:Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [167]:
#Narrow down citiesthat fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] > 80) & (city_data_df["Max Temp"] < 90) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] == 0), :]
ideal_weather_df = pd.DataFrame(ideal_weather)


# Drop any rows with null values
ideal_weather.dropna(inplace=True)


ideal_weather_df.head()

/var/folders/dp/clf5smcd4xd9krlg58kzjyfm0000gn/T/ipykernel_15270/345164161.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
233,233,nuevitas,21.5453,-77.2644,85.19,55,0,3.96,CU,1681314407
255,255,tiruchchendur,8.4833,78.1167,84.81,65,0,8.99,IN,1681314411
292,292,kattivakkam,13.2167,80.3167,86.09,68,0,8.28,IN,1681314418
300,300,buckeye,33.3703,-112.5838,82.24,25,0,6.91,US,1681314262
355,355,eldorado,-23.7869,-54.2836,81.18,63,0,5.68,BR,1681314428


### Step 3: Create a new DataFrame called `hotel_df`.

In [168]:
#Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()    


 
# Add anempty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df=hotel_df.assign(Hotel_Name="")     
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel_Name
233,nuevitas,CU,21.5453,-77.2644,55,
255,tiruchchendur,IN,8.4833,78.1167,65,
292,kattivakkam,IN,13.2167,80.3167,68,
300,buckeye,US,33.3703,-112.5838,25,
355,eldorado,BR,-23.7869,-54.2836,63,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [172]:
#Set parameters to search for a hotel
radius = 10000 
params = { Hotel_Name:"Hotel Name","radius": radius,"type": "lodging","key": geoapify_key,"limit": 1
}

#Print a message to follow up the hotel search
print("Starting hotel search")

#Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]    
    lng = row["Lng"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lat},{lng}"
    params["bias"] = f"proximity:{lat},{lng}    "
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = request.get(base_url, params=params)

    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

NameError: name 'Hotel_Name' is not defined

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
    "longitud"
    "latitud"
    geo=trrue
    tiles"OSM
    framw_width=600
    frame_height=360
    size=2
    scale=2
    color"Hotel"
    


# Display the map
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)